In [228]:
import pandas as pd

data = pd.read_csv("train_data_50_main.csv")

data['Date'] = pd.to_datetime(data['Date'])

# set indexes 
data.set_index(["Ticker", "Date"], inplace=True)

print(data.head())

                       Open      High       Low     Close  Adj Close  \
Ticker Date                                                            
ZJ     2015-07-06  5.699535  5.758382  5.695429  5.747890   5.193889   
       2015-07-07  5.742872  5.754733  5.646161  5.733748   5.181111   
       2015-07-08  5.678550  5.685849  5.590051  5.591420   5.052501   
       2015-07-09  5.649811  5.659391  5.438599  5.477374   4.949447   
       2015-07-10  5.562680  5.649811  5.529379  5.623808   5.081768   

                      Volume  
Ticker Date                   
ZJ     2015-07-06  112241600  
       2015-07-07  187787200  
       2015-07-08  243046400  
       2015-07-09  314380000  
       2015-07-10  245418000  


In [229]:
# Get the tickers
tickers = data.index.get_level_values("Ticker").unique()

# Print the sorted tickers
tickers = sorted(tickers)

tick_count = 0
corrs = []
ticker1 = []
ticker2 = []
# Want to analyze the stocks that move together or not together
for i in range(tick_count, len(tickers) - 1):
    for j in range(i + 1, len(tickers)):
        percent_change_a = (data.loc[tickers[i]]["Close"] - data.loc[tickers[i]]["Open"])/data.loc[tickers[i]]["Open"]
        a_series = pd.Series(percent_change_a)
        percent_change_b = (data.loc[tickers[j]]["Close"] - data.loc[tickers[j]]["Open"])/data.loc[tickers[j]]["Open"]
        b_series = pd.Series(percent_change_b)
        # ------FOR DEBUG PRUPOSES------
        # print(f"A_SERIES: \n{a_series}")
        # print(f"B_SERIES: \n{b_series}")
        # print(f"{tickers[i]} and {tickers[j]}: {a_series.corr(b_series)}")
        corr = a_series.corr(b_series)
        corrs.append(corr)
        ticker1.append(tickers[i])
        ticker2.append(tickers[j])

correlations = pd.DataFrame({"Ticker1": ticker1, "Ticker2": ticker2, "Correlation": corrs})
print(correlations)
correlations



     Ticker1 Ticker2  Correlation
0          A       B     0.432408
1          A       C     0.370245
2          A       D     0.452107
3          A       E     0.538456
4          A       F     0.412770
...      ...     ...          ...
1220      ZU      ZW     0.311492
1221      ZU      ZX     0.333727
1222      ZV      ZW     0.632423
1223      ZV      ZX     0.426293
1224      ZW      ZX     0.505043

[1225 rows x 3 columns]


,Ticker1,Ticker2,Correlation
0,A,B,0.432408
1,A,C,0.370245
2,A,D,0.452107
3,A,E,0.538456
4,A,F,0.412770
...,...,...,...
1220,ZU,ZW,0.311492
1221,ZU,ZX,0.333727
1222,ZV,ZW,0.632423
1223,ZV,ZX,0.426293


In [230]:
#SPX highest correlation - ZO, V, P, J, 
#DJI highest correlation - ZB


In [231]:
# Sort entire correlations df by the correlation values
correlations = correlations.sort_values(by="Correlation", ascending=False)
print(correlations)

# Capture the least and most correlated tickers
top_correlated_tickers = correlations.iloc[:6]
least_correlated_tickers = correlations.iloc[len(correlations.index) - 6:len(correlations.index)]

top_correlated_tickers = top_correlated_tickers.reset_index(drop=True)

print(least_correlated_tickers)

print(top_correlated_tickers)


     Ticker1 Ticker2  Correlation
410        J       P     0.985502
84         B      ZL     0.855175
23         A       Y     0.848811
1013      ZC      ZW     0.746992
960       ZA      ZM     0.738727
...      ...     ...          ...
1140      ZK      ZR     0.078172
1047      ZE      ZR     0.076998
867        W      ZR     0.064146
1098      ZH      ZR     0.059673
1206      ZR      ZU     0.045672

[1225 rows x 3 columns]
     Ticker1 Ticker2  Correlation
372        I       R     0.081277
1140      ZK      ZR     0.078172
1047      ZE      ZR     0.076998
867        W      ZR     0.064146
1098      ZH      ZR     0.059673
1206      ZR      ZU     0.045672
  Ticker1 Ticker2  Correlation
0       J       P     0.985502
1       B      ZL     0.855175
2       A       Y     0.848811
3      ZC      ZW     0.746992
4      ZA      ZM     0.738727
5       U      ZL     0.727735


In [232]:
import numpy as np

open_prices = []
daily_volume = []

for ticker in tickers:
    stock_close_data = data.loc[ticker]["Open"]
    open_prices.append(stock_close_data.values)

open_prices = np.stack(open_prices)

trades = np.zeros_like(open_prices)

print(open_prices[0][0])

17.042949946719865


In [233]:

min_change_percent = 0.01
positions = [0] * len(tickers)
tickers_to_idx = {k: v for v, k in enumerate(tickers)}
cash_balance = 25000

for day in range(1, len(open_prices[0])-1):
    for idx in range(len(top_correlated_tickers) - 1):
        ticker1 = tickers_to_idx[top_correlated_tickers["Ticker1"][idx]]
        ticker2 = tickers_to_idx[top_correlated_tickers["Ticker2"][idx]]
        ticker1_pct_change = (open_prices[ticker1][day] - open_prices[ticker1][day-1]) / open_prices[ticker1][day-1]
        ticker2_pct_change = (open_prices[ticker2][day] - open_prices[ticker2][day-1]) / open_prices[ticker2][day-1]
        
        # After that, determine from the dataframe which stock had a higher percent change
        # lets say A and B are correlated, if A goes up more than B goes down above a certain constant (e), buy A; if A goes up smaller than B went down, short A
        if ticker1_pct_change > 0 and ticker2_pct_change < 0 and ((ticker1_pct_change - abs(ticker2_pct_change)) > min_change_percent):
            # buy ticker 1
            if cash_balance - open_prices[ticker1][day+1] >= 0:
                trades[ticker1][day+1] = 1
                positions[ticker1] += 1
                cash_balance -= (open_prices[ticker1][day+1])

        elif ticker1_pct_change < 0 and ticker2_pct_change > 0 and ((ticker2_pct_change - (abs(ticker1_pct_change))) > min_change_percent):
            # buy ticker 2
            if cash_balance - open_prices[ticker2][day+1] >= 0:
                trades[ticker2][day+1] = 1
                positions[ticker2] += 1
                cash_balance -= (open_prices[ticker2][day+1])

        elif ticker1_pct_change > 0 and ticker2_pct_change < 0 and (ticker1_pct_change < (abs(ticker2_pct_change) - min_change_percent)):
            # short ticker 1
            trades[ticker1][day+1] = -1
            positions[ticker1] += -1

        elif ticker1_pct_change < 0 and ticker2_pct_change > 0 and (ticker2_pct_change < (abs(ticker1_pct_change) - min_change_percent)):
            # short ticker 2
            trades[ticker2][day+1] = -1
            positions[ticker2] += -1


#close shorts
for stock in range(len(positions)):
    position = positions[stock]
    if position < 0:
            trades[stock][len(open_prices[0]) - 1] += -position



In [234]:
from eval_algo import eval_actions

portfolio_value, sharpe_ratio = eval_actions(trades, open_prices, cash=25000, verbose=True)
print(f"\nPortfolio value: {portfolio_value}")
print(f"Sharpe ratio: {sharpe_ratio}")

final portfolio value: 25978.47513653701
cash: 23420.118575961114
positions: [7.0, -4.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2.0, 0, -9.0, 0, 19.0, 0, 0, 0, 0, 0, 0, 0, 0, 6.0, 12.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -19.0, 0]
short position info: defaultdict(<class 'list'>, {26: [], 48: [], 1: []})
short value: 0

Portfolio value: [25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 24999.94343321411, 24999.9014645779, 25000.12590590863, 25000.07663819758, 24999.835774296498, 24999.937959256018, 24999.91423761083, 24999.79927961838, 24999.95803136379, 24999.861320362357, 24999.91241319016, 24999.917887148247, 24999.906938535987, 24999.77738308994, 24999.636879031626, 24998.75188674163, 24999.138728659113, 24999.036543699593, 24999.259160609654, 24999.29018132968, 24999.319376932963, 24999.065739302874, 24999.281057138094, 24999.281056442018, 24998.91611244513, 24999.047491615